### 352. Data Stream as Disjoint Intervals
Given a data stream input of non-negative integers a1, a2, ..., an, summarize the numbers seen so far as a list of disjoint intervals.

Implement the SummaryRanges class:

SummaryRanges() Initializes the object with an empty stream.
void addNum(int value) Adds the integer value to the stream.
int[][] getIntervals() Returns a summary of the integers in the stream currently as a list of disjoint intervals [starti, endi]. The answer should be sorted by starti.

Example 1:<br>
Input
["SummaryRanges", "addNum", "getIntervals", "addNum", "getIntervals", "addNum", "getIntervals", "addNum", "getIntervals", "addNum", "getIntervals"]<br>
[[], [1], [], [3], [], [7], [], [2], [], [6], []]<br>
Output<br>
[null, null, [[1, 1]], null, [[1, 1], [3, 3]], null, [[1, 1], [3, 3], [7, 7]], null, [[1, 3], [7, 7]], null, [[1, 3], [6, 7]]]

Explanation
SummaryRanges summaryRanges = new SummaryRanges();<br>
summaryRanges.addNum(1);      // arr = [1]<br>
summaryRanges.getIntervals(); // return [[1, 1]]<br>
summaryRanges.addNum(3);      // arr = [1, 3]<br>
summaryRanges.getIntervals(); // return [[1, 1], [3, 3]]<br>
summaryRanges.addNum(7);      // arr = [1, 3, 7]<br>
summaryRanges.getIntervals(); // return [[1, 1], [3, 3], [7, 7]]<br>
summaryRanges.addNum(2);      // arr = [1, 2, 3, 7]<br>
summaryRanges.getIntervals(); // return [[1, 3], [7, 7]]<br>
summaryRanges.addNum(6);      // arr = [1, 2, 3, 6, 7]<br>
summaryRanges.getIntervals(); // return [[1, 3], [6, 7]]<br>

In [4]:
# time O(nlogn), space O(n)
class SummaryRanges:
    
    def __init__(self):
        self.arr = set()
        
    def addNum(self, value):
        self.arr.add(value)
        
    def getIntervals(self):
        res = []
        seen = set()
        for num in self.arr:
            if num in seen:
                continue
            left = num
            while left-1 in self.arr:
                left -= 1
                seen.add(left)
            right = num
            while right + 1 in self.arr:
                right += 1
                seen.add(right)
            res.append([left, right])
        return sorted(res)
    
ans = SummaryRanges()
ans.addNum(1)
print(ans.getIntervals())
ans.addNum(3)
print(ans.getIntervals())
ans.addNum(7)
print(ans.getIntervals())
ans.addNum(2)
print(ans.getIntervals())
ans.addNum(6)
print(ans.getIntervals())

[[1, 1]]
[[1, 1], [3, 3]]
[[1, 1], [3, 3], [7, 7]]
[[1, 3], [7, 7]]
[[1, 3], [6, 7]]


In [14]:
# binary tree search, time O(nlogn), space O(n)
from sortedcontainers import SortedList
class SummaryRanges:
    
    def __init__(self):
        self.sl = SortedList()
        
    def addNum(self, value):
        n = len(self.sl)
        i = self.sl.bisect_left([value, value])
        # value already added
        if (i > 0 and self.sl[i-1][0] <= value <= self.sl[i-1][1]) or (i < n and self.sl[i][0] <= value <= self.sl[i][1]):
            pass
        # merge left and right
        elif 0 < i < n and self.sl[i-1][1] == value - 1 and self.sl[i][0] == value + 1:
            new = [self.sl[i-1][0], self.sl[i][1]]
            del self.sl[i]
            del self.sl[i-1]
            self.sl.add(new)
        # merge left only
        elif 0 < i and self.sl[i-1][1] == value - 1:
            new = [self.sl[i-1][0], value]
            del self.sl[i-1]
            self.sl.add(new)
        # merge right only
        elif i < n and self.sl[i][0] == value + 1:
            new = [value, self.sl[i][1]]
            del self.sl[i]
            self.sl.add(new)
        else:
            self.sl.add([value, value])
        
    def getIntervals(self):
        return self.sl

ans = SummaryRanges()
ans.addNum(1)
print(ans.getIntervals())
ans.addNum(3)
print(ans.getIntervals())
ans.addNum(7)
print(ans.getIntervals())
ans.addNum(2)
print(ans.getIntervals())
ans.addNum(6)
print(ans.getIntervals())

SortedList([[1, 1]])
SortedList([[1, 1], [3, 3]])
SortedList([[1, 1], [3, 3], [7, 7]])
SortedList([[1, 3], [7, 7]])
SortedList([[1, 3], [6, 7]])


### 460. LFU Cache
Design and implement a data structure for a Least Frequently Used (LFU) cache.

Implement the LFUCache class:

LFUCache(int capacity) Initializes the object with the capacity of the data structure.
int get(int key) Gets the value of the key if the key exists in the cache. Otherwise, returns -1.
void put(int key, int value) Update the value of the key if present, or inserts the key if not already present. When the cache reaches its capacity, it should invalidate and remove the least frequently used key before inserting a new item. For this problem, when there is a tie (i.e., two or more keys with the same frequency), the least recently used key would be invalidated.
To determine the least frequently used key, a use counter is maintained for each key in the cache. The key with the smallest use counter is the least frequently used key.

When a key is first inserted into the cache, its use counter is set to 1 (due to the put operation). The use counter for a key in the cache is incremented either a get or put operation is called on it.

The functions get and put must each run in O(1) average time complexity.

Input ["LFUCache", "put", "put", "get", "put", "get", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [3], [4, 4], [1], [3], [4]]
Output [null, null, null, 1, null, -1, 3, null, -1, 3, 4]

Explanation <br>
// cnt(x) = the use counter for key x<br>
// cache=[] will show the last used order for tiebreakers (leftmost element is  most recent)<br>
LFUCache lfu = new LFUCache(2);<br>
lfu.put(1, 1);   // cache=[1,_], cnt(1)=1<br>
lfu.put(2, 2);   // cache=[2,1], cnt(2)=1, cnt(1)=1<br>
lfu.get(1);      // return 1<br>
                 // cache=[1,2], cnt(2)=1, cnt(1)=2<br>
lfu.put(3, 3);   // 2 is the LFU key because cnt(2)=1 is the smallest, invalidate 2.<br>
                 // cache=[3,1], cnt(3)=1, cnt(1)=2<br>
lfu.get(2);      // return -1 (not found)<br>
lfu.get(3);      // return 3<br>
                 // cache=[3,1], cnt(3)=2, cnt(1)=2<br>
lfu.put(4, 4);   // Both 1 and 3 have the same cnt, but 1 is LRU, invalidate 1.<br>
                 // cache=[4,3], cnt(4)=1, cnt(3)=2<br>
lfu.get(1);      // return -1 (not found)<br>
lfu.get(3);      // return 3<br>
                 // cache=[3,4], cnt(4)=1, cnt(3)=3<br>
lfu.get(4);      // return 4<br>
                 // cache=[4,3], cnt(4)=2, cnt(3)=3<br>

In [35]:
from collections import deque, defaultdict

class Node:
    
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.freq = 1

class LFUCache:
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.minFreq = 0
        self.keyToNode = dict() # map 1, srore key-value pair
        self.freqToList = defaultdict(deque) # map 2, store key-frequency pair 
#         self.freqToKey = defaultdict(set) # map 3, store frequency-key pair
        
    def get(self, key):
        if key not in self.keyToNode:
            return -1
        # in the cecha, update frequency
        node = self.keyToNode[key]
        self.updateFreq(node)
        return node.value
    
    def put(self, key, value):
        if self.capacity == 0: # corner case
            return 
        if key in self.keyToNode: # key in cache, update value associated to key, update frequency
            node = self.keyToNode[key]
            node.value = value # update value
            self.updateFreq(node) # update frequency
            return 
        if len(self.keyToNode) == self.capacity: # if key not in cache and cache is full
            keyToRemove = self.freqToList[self.minFreq].pop() # pop least freq key from map freqToList
#             self.freqToKey[self.minFreq].remove(keyToRemove) # remove least freq key from map freqToKey
            del self.keyToNode[keyToRemove] # remove least freq key from map keyToNode
        self.minFreq = 1 # reset minFreq to 1, because of putting a new key
        self.freqToList[1].append(key) # add the new key to the map freqToList with frequency of 1
#         self.freqToKey[1].add(key) # add the new key to the map freqToKey with frequency of 1
        self.keyToNode[key] = Node(key, value) # add the new key to the map keyToNode
    
    def updateFreq(self, node):
        prevFreq = node.freq
        newFreq = node.freq + 1
        self.freqToList[prevFreq].remove(node.key) # remove the key from the current freq
#         self.freqToKey[prevFreq].remove(node.key) # remove the key from the current freq
        if len(self.freqToList[prevFreq]) == 0: # if the current freq has no key left
            del self.freqToList[prevFreq] # remove the freq-key pair
            if prevFreq == self.minFreq: # update minFreq if it is the current freq
                self.minFreq += 1
        if newFreq not in self.freqToList: # if the updated freq not in the maps, initialise it
            self.freqToList[newFreq] = deque()
#             self.freqToKey[newFreq].add(node.key)
        # update the new freq in all three maps
        self.freqToList[newFreq].appendleft(node.key)
#         self.freqToKey[newFreq].add(node.key)
        node.freq = newFreq
        
ans = LFUCache(2)
ans.put(1, 1)
ans.put(2, 2)
print(ans.get(1))
ans.put(3, 3)
print(ans.get(2))
print(ans.get(3))
ans.put(4, 4)
print(ans.get(1))
print(ans.get(3))
print(ans.get(4))

1
-1
3
-1
3
4


In [40]:
# DLL and dictionary
from collections import defaultdict

class ListNode:
    
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.freq = 1
        self.prev = None
        self.next = None
        
        
class DLL: # double linked list
    
    def __init__(self):
        self.head = ListNode(0, 0)
        self.tail = ListNode(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head
        self.size = 0
        
    def insertHead(self, node): # insert the new node to the head of the map when a new key is pushed to cache
        # insert new node to the right of head
        headNext = self.head.next
        headNext.prev = node
        self.head.next = node
        node.prev = self.head
        node.next = headNext
        self.size += 1
        
    def removeNode(self, node):
        node.next.prev = node.prev
        node.prev.next = node.next
        self.size -= 1
        
    def removeTail(self): # remove tail node when cache is full
        # remove node at the left of the tail
        tail = self.tail.prev
        self.removeNode(tail)
        return tail
    
    
class LFUCache:
    
    def __init__(self, capacity):
        self.capacity = capacity
        # freaTable: a dictionary to store the mapping of diffrent frequency values with values
        # as DLLs storing (key, value) pairs as nodes
        self.freqTable = defaultdict(DLL)
        # cache dictionary: nodes in the DLL are stored as values for each key pushed into the cache
        self.cache = {}
        self.minFreq = 0
        
    def get(self, key):
        if key not in self.cache:
            return -1
        return self.updateCache(self.cache[key], key, self.cache[key].value)
    
    def put(self, key, value):
        if self.capacity == 0:
            return
        if key in self.cache:
            self.updateCache(self.cache[key], key, value)
        else:
            if len(self.cache) == self.capacity:
                prevTail = self.freqTable[self.minFreq].removeTail()
                del self.cache[prevTail.key]
            node = ListNode(key, value)
            self.freqTable[1].insertHead(node)
            self.cache[key] = node
            self.minFreq = 1
            
    def updateCache(self, node, key, value):
        node = self.cache[key]
        node.value = value # update node value
        # update node frequency and update freqTable
        prevFreq = node.freq 
        node.freq += 1
        self.freqTable[prevFreq].removeNode(node) # remove node from prevFreq
        self.freqTable[node.freq].insertHead(node) # inseart node to the updated freq
        if prevFreq == self.minFreq and self.freqTable[prevFreq].size == 0: # update minFreq
            self.minFreq += 1
        return node.value
    
ans = LFUCache(2)
ans.put(1, 1)
ans.put(2, 2)
print(ans.get(1))
ans.put(3, 3)
print(ans.get(2))
print(ans.get(3))
ans.put(4, 4)
print(ans.get(1))
print(ans.get(3))
print(ans.get(4))

1
-1
3
-1
3
4
